- Import library and read data with pandas 

In [ ]:
import pandas as pd
CO2Data = pd.read_csv("../data/Terminos_lagoon_TA_DIC__2023_RawData.csv")

### Scatter plot  with linear regretion = scipy.stats.linregress

A simple and quick function for performing linear regression on two variables. It calculates the slope, intercept, R-squared, p-value, and standard error. Best for basic, exploratory analysis with one independent variable.

In [ ]:
import matplotlib.pyplot as plt
from scipy import stats

x = CO2Data['ta_micromol_kg']
y = CO2Data['dic_micromol_kg']

plt.scatter(x, y, label='original data')

# Add labels and title
plt.xlabel('TA ($\mu mol  \; kg^{-1}$)', fontsize = 12, )
plt.ylabel('DIC ($\mu mol  \; kg^{-1}$)', fontsize = 12)

# Calculate the linear regression line
slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

# Plot linear regression 

plt.plot(x, intercept + slope*x, 'r', label='fitted line')

# set the figure size
plt.gcf().set_size_inches(6, 4)

# save the plot as a PDF file paper format 

plt.savefig('DIC_TA_pH.pdf', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
print("r-squared:", r_value**2)
print("p_value:", p_value)

### Perform  least squares (OLS) regression: scipy.stats.linregress

A powerful and detailed linear regression tool that supports both simple and multiple linear regression. It provides a full statistical report, including p-values, confidence intervals, and diagnostic tests. Ideal for professional data analysis and scientific reporting.

In [ ]:
import statsmodels.api as sm
import numpy as np

#define predictor and response variables
x = CO2Data['ta_micromol_kg']
y = CO2Data['dic_micromol_kg']

#add constant to predictor variables
x = sm.add_constant(x)

#fit linear regression model
model = sm.OLS(y, x).fit()

#view model summary
print(model.summary())



# --- Interpretación automática ---
print("\n--- Interpretation of OLS Results ---\n")

# R-squared
r2 = model.rsquared
print(f"R-squared: {r2:.3f}")
if r2 > 0.7:
    print("➡️ The model explains a large proportion of the variance.")
elif r2 > 0.4:
    print("➡️ The model explains a moderate proportion of the variance.")
else:
    print("➡️ The model explains a low proportion of the variance. Consider reviewing your predictors.")

# Coefficients and p-values
print("\nCoefficients and p-values:")
summary_table = model.summary2().tables[1]
for index, row in summary_table.iterrows():
    coef = row['Coef.']
    pval = row['P>|t|']
    print(f"- {index}: Coef = {coef:.4f}, p-value = {pval:.4f}")
    if pval < 0.05:
        print("  ✅ Statistically significant (p < 0.05)")
    else:
        print("  ⚠️ Not statistically significant (p >= 0.05)")

# Residuals check
print("\nStandard Error of the regression (SE): {:.4f}".format(np.sqrt(model.scale)))
print("➡️ Lower SE indicates better model fit, but check in context of your data scale.")

# export model summary to text file
with open('model_summary.txt', 'w') as file:
    file.write(model.summary().as_text())

In [ ]:


# --- Interpretación automática ---# Interpretación simplificada
print("\n=== MODEL INTERPRETATION ===\n")

# R-squared
r2 = model.rsquared
print(f"R² = {r2:.3f}")
if r2 >= 0.7:
    print("✔️ Good model fit: Explains most of the variance.")
elif r2 >= 0.4:
    print("⚠️ Moderate model fit: Explains part of the variance.")
else:
    print("❌ Weak model fit: Explains little variance. Review your model.")

# Extract coefficients and p-values
results = model.summary2().tables[1]

print("\nCoefficients:")
for var, row in results.iterrows():
     pval = row['P>|t|']
     coef = row['Coef.']
     significance = "✔️ Significant (p < 0.05)" if pval < 0.05 else "⚠️ Not significant (p ≥ 0.05)"
     print(f"- {var}: Coef = {coef:.4f}, p = {pval:.4f} → {significance}")

# Standard Error
se = np.sqrt(model.scale)
print(f"\nStandard Error of the model: {se:.4f}")
print("Lower SE indicates better precision, check relative to your data scale.")

In [ ]:
print("\n=== MODEL INTERPRETATION ===\n")

# R-squared
r2 = model.rsquared
print(f"R² = {r2:.3f}")
fit_quality = "✔️ Good model fit" if r2 >= 0.7 else "⚠️ Moderate model fit" if r2 >= 0.4 else "❌ Weak model fit"
print(f"{fit_quality}: Explains {'most' if r2 >= 0.7 else 'part' if r2 >= 0.4 else 'little'} of the variance.")

# Extract coefficients and p-values
results = model.summary2().tables[1]
print("\nCoefficients:")
for var, row in results.iterrows():
    coef, pval = row['Coef.'], row['P>|t|']
    significance = "✔️ Significant (p < 0.05)" if pval < 0.05 else "⚠️ Not significant (p ≥ 0.05)"
    print(f"- {var}: Coefficient = {coef:.4f}, p = {pval:.4f} → {significance}")

# Intercept and slope
intercept_coef, intercept_pval = results.loc['const', ['Coef.', 'P>|t|']]
slope_var = results.index.drop('const')[0]  # assuming one predictor
slope_coef, slope_pval = results.loc[slope_var, ['Coef.', 'P>|t|']]

print(f"\nIntercept (const): {intercept_coef:.4f}, p = {intercept_pval:.4f} → "
      f"{'✔️ Significant' if intercept_pval < 0.05 else '⚠️ Not significant'}")
print(f"Slope ({slope_var}): {slope_coef:.4f}, p = {slope_pval:.4f} → "
      f"{'✔️ Significant' if slope_pval < 0.05 else '⚠️ Not significant'}")

# Standard Error
print(f"\nStandard Error of the model: {np.sqrt(model.scale):.4f}")